In [124]:
%%writefile train.py

import pandas as pd
import numpy as np

Overwriting train.py


In [125]:
'''
指标构造—python code
exit_it  = 行业平均固定资产/市场平均固定资产 + 行业平均投资/市场总投资 + 行业平均价格/市场总体价格 + 行业平均资产负债率/市场总负债率 + 行业平均利润率/市场总利润率
i为行业,t为年份
'''

'\n指标构造—python code\nexit_it  = 行业平均固定资产/市场平均固定资产 + 行业平均投资/市场总投资 + 行业平均价格/市场总体价格 + 行业平均资产负债率/市场总负债率 + 行业平均利润率/市场总利润率\ni为行业,t为年份\n'

In [126]:
df = pd.read_stata('/Users/wangjunjie/Desktop/大二下/大二下课程/产业分析/小组/project2/data/data_98_07.dta')

id_i = pd.to_numeric(df['id1_indus'], errors='coerce')
state_open = pd.to_numeric(df['state'], errors='coerce')

df['id_indus_1'] = id_i
df['state_open'] = state_open

In [127]:
df["state_cal"] = df['state_open'].apply(lambda x: 1 if x == 1.0 else 0)
exit_rate = df.groupby(['year', 'id_indus_1'])["state_cal"].mean()
exit_rate = pd.DataFrame(1-exit_rate)
exit_rate['exit_rate'] = exit_rate['state_cal']
exit_rate = exit_rate.drop('state_cal', axis = 1)

In [128]:
#计算entry_rate
df['year_working'] = df['year'] - df['year_open']
df["entry_rate"] = df['year_working'].apply(lambda x: 1 if x <= 2.0 else 0)
entry_rate = pd.DataFrame(df.groupby(['year', 'id_indus_1'])["entry_rate"].mean())

In [129]:
df_data = pd.merge(entry_rate, exit_rate, on = ['year','id_indus_1'])
#df_data.to_excel('industry.xlsx')

In [130]:
df_data['industry_id'] = df_data.index.get_level_values(1)
df_data['year_id'] = df_data.index.get_level_values(0)

#invest of particular industry
mean_invest = pd.DataFrame(df.groupby(['year', 'id_indus_1'])['log_invest'].mean())
mean_invest['mean_invest'] = mean_invest['log_invest']
mean_invest = mean_invest.drop('log_invest',axis = 1)
df_data = pd.merge(df_data, mean_invest, on = ['year','id_indus_1'])

# invest of all industry
df_invest_all = pd.DataFrame(df.groupby('year')['log_invest'].mean())
df_data = pd.merge(df_data, df_invest_all, on = ['year'], right_index = True)
df_data['invest_mean_all'] = df_data['log_invest']
df_data = df_data.drop('log_invest',axis = 1)

In [131]:
#invest of particular industry
mean_fix_invest = pd.DataFrame(df.groupby(['year', 'id_indus_1'])['fix_assets'].mean())
mean_fix_invest['mean_fix_assets'] = mean_fix_invest['fix_assets']
mean_fix_invest = mean_fix_invest.drop('fix_assets',axis = 1)
df_data = pd.merge(df_data, mean_fix_invest, on = ['year','id_indus_1'])

# invest of all industry
df_fix_invest_all = pd.DataFrame(df.groupby('year')['fix_assets'].mean())

df_fix_invest_all['fix_invest_mean_all'] = df_fix_invest_all['fix_assets']
df_fix_invest_all = df_fix_invest_all.drop('fix_assets',axis = 1)

df_data = pd.merge(df_data, df_fix_invest_all, on = ['year'], right_index = True)
# show df_data

In [132]:
#invest of particular industry
mean_price = pd.DataFrame(df.groupby(['year', 'id_indus_1'])['acc_receivable'].mean())
mean_price['acc_receivable_indus'] = mean_price['acc_receivable']
mean_price = mean_price.drop('acc_receivable',axis = 1)

df_data = pd.merge(df_data, mean_price, on = ['year','id_indus_1'])

In [133]:
# invest of all industry
df_price_all = pd.DataFrame(df.groupby('year')['acc_receivable'].mean())
df_price_all['acc_receivable_indus_all'] = df_price_all['acc_receivable']
df_price_all = df_price_all.drop('acc_receivable',axis = 1)

df_data = pd.merge(df_data, df_price_all, on = ['year'], right_index = True)

In [134]:
#asset_liability有很多值不存在（分母为0）
#df['asset_lia'] = df['assets'] / df['liability']

mean_al = pd.DataFrame(df.groupby(['year', 'id_indus_1'])['assets'].mean())
mean_al['ass_indus'] = mean_al['assets']
mean_al = mean_al.drop('assets',axis = 1)

df_data = pd.merge(df_data, mean_al, on = ['year','id_indus_1'], right_index = True)


df_al = pd.DataFrame(df.groupby('year')['assets'].mean())
df_al['ass_all'] = df_al['assets']
df_al = df_al.drop('assets',axis = 1)

df_data = pd.merge(df_data, df_al, on = ['year'], right_index = True)

In [135]:
mean_income = pd.DataFrame(df.groupby(['year', 'id_indus_1'])['income_main'].mean())
mean_income['income_indus'] = mean_income['income_main']
mean_income = mean_income.drop('income_main',axis = 1)

df_data = pd.merge(df_data, mean_income, on = ['year','id_indus_1'], right_index = True)

df_income_all = pd.DataFrame(df.groupby('year')['income_main'].mean())
df_income_all['income_indus_all'] = df_income_all['income_main']
df_income_all = df_income_all.drop('income_main',axis = 1)

df_data = pd.merge(df_data, df_income_all, on = ['year'], right_index = True)

mean_cost = pd.DataFrame(df.groupby(['year', 'id_indus_1'])['cost_main'].mean())
mean_cost['cost_indus'] = mean_cost['cost_main']
mean_cost = mean_cost.drop('cost_main',axis = 1)

df_data = pd.merge(df_data, mean_cost, on = ['year','id_indus_1'], right_index = True)

df_cost = pd.DataFrame(df.groupby('year')['cost_main'].mean())
df_cost['cost_main_al'] = df_cost['cost_main']
df_cost = df_cost.drop('cost_main',axis = 1)

df_data = pd.merge(df_data, df_cost, on = ['year'], right_index = True)


In [136]:
mean_profit = pd.DataFrame(df.groupby(['year', 'id_indus_1'])['material'].mean())
mean_profit['material_indus'] = mean_profit['material']
mean_profit = mean_profit.drop('material',axis = 1)

df_data = pd.merge(df_data, mean_profit, on = ['year','id_indus_1'], right_index = True)

df_profit_all = pd.DataFrame(df.groupby('year')['material'].mean())
df_profit_all['material_indus_all'] = df_profit_all['material']
df_profit_all = df_profit_all.drop('material',axis = 1)

df_data = pd.merge(df_data, df_profit_all, on = ['year'], right_index = True)
df_data.to_excel('industry_data_processing.xlsx')

In [137]:
'''
df[‘profit_rate’] = df[‘profit’] / df[‘cost’]
行业平均利润率 = df.groupby(year, id_indus)[‘profit_rate’].mean()
市场总利润率 = df.groupby(year)[‘profit_rate’].mean()
'''

'\ndf[‘profit_rate’] = df[‘profit’] / df[‘cost’]\n行业平均利润率 = df.groupby(year, id_indus)[‘profit_rate’].mean()\n市场总利润率 = df.groupby(year)[‘profit_rate’].mean()\n'

In [138]:
df_wall = pd.read_excel('/Users/wangjunjie/Desktop/大二下/大二下课程/产业分析/小组/project2/data/industry_data_processing.xlsx')
df_wall['cap_sale'] = df_wall['mean_fix_assets'] / df_wall['income_indus']

In [139]:
min_scale = pd.DataFrame(df.groupby(['year','id_indus_1'])['assets'].median())
min_scale['year_id'] = min_scale.index.get_level_values(0)
min_scale['industry_id'] = min_scale.index.get_level_values(1)
min_scale_1 = min_scale
min_scale_1['assets_med'] = min_scale['assets']

min_scale_1 = min_scale_1.drop('assets',axis = 1)

In [140]:
df_cal_wall = pd.merge(df_wall, min_scale_1, on = ['year_id','industry_id'])

In [141]:
df_cal_wall['entry_sunk_cost'] = df_cal_wall['cap_sale']*df_cal_wall['assets_med']
df_cal_wall.to_excel('industry_data_6_10.xlsx')